In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json

In [ ]:
df_races = pd.read_json("racesfr.json")
trPeople = pd.read_json("trSailorInfoAll.json")
df_races

In [59]:
# Filter people without a link in df_races
df_races['Link'] = df_races['Link'].fillna('Unknown') # fill empty links

df_races_no_link = df_races[df_races['Link'] == 'Unknown']

# Filter people without a link in trPeople
trPeople_no_link = trPeople[trPeople['link'] == 'Unknown']

# Combine the data from both sources (people without a link)
df_no_link = pd.concat([
    df_races_no_link[['Sailor', 'GradYear', 'Team']].rename(columns={'Sailor': 'name', 'GradYear': 'year', 'Team': 'team'}),
    trPeople_no_link[['key', 'name', 'year', 'team']]
])

df_no_link['key'] = df_no_link['key'].fillna(df_no_link['name'] + '-' + df_no_link['team'])

print("Kyle Cobbs-Texas" in list(df_no_link['key']))
print(len(list(df_no_link['key'].unique())))

True
1694


In [ ]:
links = df_races['Link'].unique()
links = np.append(links, trPeople['link'].unique())
links = links[links != None]
links = links[links != 'Unknown']
links = np.unique(links)

In [37]:
allRows = []
for link in links[:5]:
    url = f"https://scores.collegesailing.org/sailors/{link}/"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    sailorData = json.loads(soup.find('meta', attrs={'name': 'ts:data'})['content'])
    school = soup.find('ul',id='menu').contents[3].text
    allRows.append([link, sailorData['name'],sailorData['first_name'],sailorData['last_name'],sailorData['gender'],sailorData['year'],sailorData['school'].split("/")[2],school, sailorData['id'], sailorData['external_id']])
    
df_sailor_info = pd.DataFrame(allRows, columns=['link', 'name', 'first_name', 'last_name', 'gender', 'year', 'schoollink','team','id', 'external_id'])
df_sailor_info

,link,name,first_name,last_name,gender,year,schoollink,team,id,external_id
0,aadvika-ahuja,aadvika ahuja,aadvika,ahuja,F,2025,uc-davis,UC Davis,3136210,None
1,aaron-blust,Aaron Blust,Aaron,Blust,M,2024,olin,Olin,3127882,None
2,aaron-burnett,Aaron Burnett,Aaron,Burnett,M,2019,florida,Florida,3114274,27657
3,aaron-comen,Aaron Comen,Aaron,Comen,M,2018,texas,Texas,3109429,25596
4,aaron-fairchild,Aaron Fairchild,Aaron,Fairchild,M,2021,pittsburgh,U Pittsburgh,3120599,None


In [ ]:
sailor_info = pd.read_json("sailor_data2.json").dropna()
sailor_info

,link,name,first_name,last_name,gender,year,school,id,external_id
0,campbell-deliscu,Campbell D'Eliscu,Campbell,D'Eliscu,M,2019.0,georgetown,3112221.0,27830.0
1,sean-segerblom,Sean Segerblom,Sean,Segerblom,M,2020.0,georgetown,3117220.0,29802.0
2,meaghan-macrae,Meaghan MacRae,Meaghan,MacRae,F,2018.0,georgetown,3109481.0,25661.0
3,claire-mohun,Claire Mohun,Claire,Mohun,F,2019.0,georgetown,3115481.0,28785.0
4,steven-leuck,Steven Leuck,Steven,Leuck,M,2020.0,charleston,3117488.0,30344.0
...,...,...,...,...,...,...,...,...,...
12142,cwhitesell,Chloe Whitesell,Chloe,Whitesell,F,2028.0,citadel,277047.0,14551.0
12589,bradlee,Bradlee Anderson,Bradlee,Anderson,M,2028.0,citadel,277046.0,12183.0
12688,bhhrrrr,Hannah \u201cSic \u2018Em\u201d Wickham,Hannah \u201cSic \u2018Em\u201d,Wickham,F,2027.0,citadel,3108690.0,24339.0
13180,zestyhesty,Thomas Hester,Thomas,Hester,M,2027.0,citadel,277042.0,14527.0


In [ ]:
js = """{'version':'1.0','external_id':null,'first_name':'Catherine \\"B\\",'gender':'F','id':'3124222','last_name':'Lindsay','name':'Catherine \"B\" Lindsay','school':'url:/schools/yale/','year':'2023'}"""
sailor_data = {'gender': "F", 'name': 'Catherine "B" Lindsay', 'first_name':'Catherine "B"', 'last_name':'Lindsay', 'year': '2023', 'school': 'url:/schools/yale/', 'id': '3124222', 'external_id': None}
sailor_data2 = {"name": 'Iona Deacon', 'year': '2016', 'school': '/schools/british-columbia/', 'gender': 'F'}
sailor_data3 = {"name": 'Henriette Smith', 'year': '2016', 'school': '/schools/oregon/', 'gender': 'F'}

json.loads(js)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)